In [17]:
import numpy as np, pandas as pd
import time, random, json, tqdm, re, csv
%matplotlib inline

In [18]:
pd.options.display.max_columns=None
pd.options.display.max_rows=250

# Kriminalitets data
Data scrapet fra https://statistik.politi.dk

In [19]:
crime_1=pd.read_csv("Data(fra Mikkel)/Kriminalitet/drab,brand,mv..csv", delimiter=";")
crimetemp1=pd.DataFrame()
crimetemp1["Municipality"]=crime_1["Kommune"]
crimetemp1["Murder&Arson_reported"]=crime_1["Antal anmeldelser"]
crimetemp1["Murder&Arson_charged"]=crime_1["Antal sigtelser"]

crime_2=pd.read_csv('Data(fra Mikkel)/Kriminalitet/indbrud.csv', delimiter=";")
crimetemp2=pd.DataFrame()
crimetemp2["Municipality"]=crime_2["Kommune"]
crimetemp2["Breaking&entering_reported"]=crime_2["Antal anmeldelser"]
crimetemp2["Breaking&entering_charged"]=crime_2["Antal sigtelser"]

crime_3=pd.read_csv('Data(fra Mikkel)/Kriminalitet/narkotika.csv', delimiter=";")
crimetemp3=pd.DataFrame()
crimetemp3["Municipality"]=crime_3["Kommune"]
crimetemp3["Narcotics_reported"]=crime_3["Antal anmeldelser"]
crimetemp3["Narcotics_charged"]=crime_3["Antal sigtelser"]

crime_4=pd.read_csv('Data(fra Mikkel)/Kriminalitet/røveri.csv', delimiter=";")
crimetemp4=pd.DataFrame()
crimetemp4["Municipality"]=crime_4["Kommune"]
crimetemp4["Robbery_reported"]=crime_4["Antal anmeldelser"]
crimetemp4["Robbery_charged"]=crime_4["Antal sigtelser"]

crime_5=pd.read_csv('Data(fra Mikkel)/Kriminalitet/sædelighed.csv', delimiter=";")
crimetemp5=pd.DataFrame()
crimetemp5["Municipality"]=crime_5["Kommune"]
crimetemp5["Indecency_reported"]=crime_5["Antal anmeldelser"]
crimetemp5["Indecency_charged"]=crime_5["Antal sigtelser"]

crime_6=pd.read_csv('Data(fra Mikkel)/Kriminalitet/tyveri.csv', delimiter=";")
crimetemp6=pd.DataFrame()
crimetemp6["Municipality"]=crime_6["Kommune"]
crimetemp6["Theft_reported"]=crime_6["Antal anmeldelser"]
crimetemp6["Theft_charged"]=crime_6["Antal sigtelser"]

crime_7=pd.read_csv('Data(fra Mikkel)/Kriminalitet/vold.csv', delimiter=";")
crimetemp7=pd.DataFrame()
crimetemp7["Municipality"]=crime_7["Kommune"]
crimetemp7["Assault_reported"]=crime_7["Antal anmeldelser"]
crimetemp7["Assault_charged"]=crime_7["Antal sigtelser"]

crimelist=[crimetemp1, crimetemp2, crimetemp3, crimetemp4, crimetemp5, crimetemp6, crimetemp7]
crimelist=[df.set_index("Municipality") for df in crimelist]
crime=crimelist[0].join(crimelist[1:], how="outer", sort=True)
crime=crime.fillna(value=0)

crime["Total_reported"]=crime["Murder&Arson_reported"]+crime["Breaking&entering_reported"]+crime["Narcotics_reported"]\
                        +crime["Robbery_reported"]+crime["Indecency_reported"]+crime["Theft_reported"]+crime["Assault_reported"]
crime["Total_charged"]=crime["Murder&Arson_charged"]+crime["Breaking&entering_charged"]+crime["Narcotics_charged"]\
                        +crime["Robbery_charged"]+crime["Indecency_charged"]+crime["Theft_charged"]+crime["Assault_charged"]

crime.to_csv("Crime")

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6848: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  verify_integrity=True)


# Etnicitet
Data scrapet fra statisitkbanken (FOLK1E)

In [20]:
data_xls = pd.read_excel('DST_Excel/FOLK1E.xlsx', 'FOLK1E', index_col=None)
data_xls.to_csv('FOLK1E.csv', encoding='utf-8', index=False)
etnodata=pd.read_csv("FOLK1E.csv")
etnocrime= etnodata.join(crime,on="Municipality").fillna(method="pad")

# Social & Indenrigsministeriets kommunale nøgletal.
Hentet fra http://www.noegletal.dk/

In [21]:
SIData = pd.read_excel("Data(fra Mikkel)/Nøgletal/Nøgletal.xlsx", "Nøgletal",index_col=None)
SIData.to_csv("SI_data.csv", encoding='utf-8', index=False)
soecData=pd.read_csv("SI_data.csv")
soecData.set_index("Municipality")

findata=pd.concat([etnocrime.reset_index(drop=True),soecData],axis=1)
findata.set_index("Municipality",verify_integrity=True,drop=True)
findata
df = findata.loc[:,~findata.columns.duplicated()]

In [22]:
temp=pd.read_csv("Boliga - Behandling 3 - Oprydning.csv")

final=temp.merge(df,how="left",left_on=["Kommune"],right_on=["Municipality"],validate="m:1")

final

,basementSize,buildYear,city,createdDate,daysForSale,Ejerudgift,floor,id,isForeclosure,latitude,longitude,lotSize,municipality,price,rooms,size,street,zipCode,coordinate,Kommune,Region,Type,Befolkning,Relativ Ledighed,Grundskole,Gymnasiale uddannelser,Erhvervsfaglige uddannelser,KVU,MVU,Bacheloruddannelser,LVU,Kommunal_gennemsnitsinkomst_2017,Municipality,danish_origin,immigrants_Western,immigrants_Non_Western,descendants_Western,descendants_Non_Western,Murder&Arson_reported,Murder&Arson_charged,Breaking&entering_reported,Breaking&entering_charged,Narcotics_reported,Narcotics_charged,Robbery_reported,Robbery_charged,Indecency_reported,Indecency_charged,Theft_reported,Theft_charged,Assault_reported,Assault_charged,Total_reported,Total_charged,Population_density(pop/km2),Population_in_urban_development,Socioeconomic_index,expenditure_per_capita,Service_expenses_per_capita,church_tax,land_tax(‰),land_value_per_capita,taxable_income_per_capita,expenses_development,expenses_transport&infrastructure,expenses_public_school_per_capita,expenses_daycare_per_capita,long_term_debt_per_capita,liquid_assets_per_capita,expenses_daycare_per_0-13yo(brutto),expenses_daycare_per_0-13yo(netto),monthly_price_daycare(3-5yo),education_expenses(6-16yo),expenses_per_school_student,number_of_public_schools,number_of_ordinary_classes,average_class_size,library_expenses_per_capita,expenses_sport_and_other_cultural_activities,expenses_elder(+67yo),sickness_benefits_per_17-64yo,expenses_for_assistance(17-64yo),expenses_for_reschooling(17-64yo)
0,0,1767,København K,2019-05-22T00:35:07.000Z,93,16872,3.0,1559151,False,55.67996,12.58478,0,101,18750000,6.0,280,"Kongens Nytorv 21, 3. tv",1050,"55.679959999999994, 12.58478",København,Region Hovedstaden,Ejerlejlighed,626508,7.926682,13.754726,7.646977,19.268325,4.928181,17.977495,3.627649,26.658495,319745,København,466883,57587,65111,6196,30731,67.0,59.0,1.209,103.0,2.622,2.129,128.0,70.0,123.0,86.0,6.318,1.158,850.0,588.0,1178.149,909.287,7190,100.0,1.08,58777,41385,0.80,34.00,190274,195564,1518,417,6276,7197,2310,15618,66494,49568,1700,66679,69710,60.0,1708,22.8,346,1062,64106,1465,4147,1493
1,88,1880,København K,2019-06-14T03:27:01.490Z,70,5702,0.0,1567291,False,55.67926,12.59149,0,101,10500000,4.0,83,"Nyhavn 38, st. Dør/lejl. 4",1051,"55.67926, 12.59149",København,Region Hovedstaden,Ejerlejlighed,626508,7.926682,13.754726,7.646977,19.268325,4.928181,17.977495,3.627649,26.658495,319745,København,466883,57587,65111,6196,30731,67.0,59.0,1.209,103.0,2.622,2.129,128.0,70.0,123.0,86.0,6.318,1.158,850.0,588.0,1178.149,909.287,7190,100.0,1.08,58777,41385,0.80,34.00,190274,195564,1518,417,6276,7197,2310,15618,66494,49568,1700,66679,69710,60.0,1708,22.8,346,1062,64106,1465,4147,1493
2,0,1755,København K,2019-04-24T00:09:51.880Z,121,9486,1.0,1549766,False,55.67981,12.59206,0,101,15000000,7.0,274,"Nyhavn 53, 1.",1051,"55.679809999999996, 12.59206",København,Region Hovedstaden,Ejerlejlighed,626508,7.926682,13.754726,7.646977,19.268325,4.928181,17.977495,3.627649,26.658495,319745,København,466883,57587,65111,6196,30731,67.0,59.0,1.209,103.0,2.622,2.129,128.0,70.0,123.0,86.0,6.318,1.158,850.0,588.0,1178.149,909.287,7190,100.0,1.08,58777,41385,0.80,34.00,190274,195564,1518,417,6276,7197,2310,15618,66494,49568,1700,66679,69710,60.0,1708,22.8,346,1062,64106,1465,4147,1493
3,0,1880,København K,2019-05-08T00:04:30.737Z,107,17609,5.0,1554276,False,55.67926,12.59149,0,101,19995000,7.0,296,"Nyhavn 38, 5. tv.",1051,"55.67926, 12.59149",København,Region Hovedstaden,Ejerlejlighed,626508,7.926682,13.754726,7.646977,19.268325,4.928181,17.977495,3.627649,26.658495,319745,København,466883,57587,65111,6196,30731,67.0,59.0,1.209,103.0,2.622,2.129,128.0,70.0,123.0,86.0,6.318,1.158,850.0,588.0,1178.149,909.287,7190,100.0,1.08,58777,41385,0.80,34.00,190274,195564,1518,417,6276,7197,2310,15618,66494,49568,1700,66679,69710,60.0,1708,22.8,346,1062,64106,1465,4147,1493
4,0,1873,København K,2019-07-12T03:21:26.060Z,42,5590,3.0,1576691,False,55.67852

In [23]:
#Drop excess columns
final = final.drop(columns = ['Municipality'])

In [24]:
final.to_csv("Boliga - Behandling 4 - Med etnosociodata.csv", index = False)